In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uptrain
import cv2
import tensorflow as tf
from tensorflow import keras
import os

In [2]:
def load_images(path):
    list_dir = os.listdir(path)
    img,label = [],[]
    for i in list_dir:
        image_dir = os.listdir(path+i)
        for paths in image_dir:
            image = cv2.cvtColor(cv2.imread(path+i+"\\"+paths),cv2.COLOR_BGR2GRAY)
            img.append(image)
            label.append(int(i))
    return np.array(img),np.array(label)


In [3]:
img,labels = load_images("C:\\Users\\Lenovo\\OneDrive\\Desktop\\MNIST USING Uptrain\\MNIST Dataset\\trainingSet\\")

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Dropout,Flatten
model = Sequential([
    Conv2D(128,(3,3),activation = 'relu',input_shape = (28,28,1)),
    Conv2D(64,(3,3),activation='relu'),
    Conv2D(32,(3,3),activation='relu'),
    Flatten(),
    Dense(128,activation = 'relu'),
    Dense(64,activation = 'relu'),
    Dense(32,activation = 'relu'),
    Dense(10,activation = 'softmax')
])
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        73792     
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 22, 32)        18464     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 128)               1982592   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2

In [6]:
from sklearn.model_selection import train_test_split
img = img.reshape(42000,28,28,1)
labels = labels.reshape(42000,1)
Xtrain,Xtest,ytrain,ytest = train_test_split(img,labels,test_size = 0.2,shuffle=True)

In [10]:
import json
train_data = []
for i in range(len(Xtrain)):
    train_data.append({
         "kps": Xtrain[i].tolist(),
         "label": ytrain[i].tolist()
     })
test_data = []
for i in range(len(Xtest)):
    test_data.append({
         "kps": Xtest[i].tolist(),
         "label": ytest [i].tolist()
     })
with open('train_set.json','w') as fp:
    json.dump(train_data,fp)
with open('test_set.json','w') as fp1:
    json.dump(test_data,fp1)

In [11]:
for i in range(len(Xtrain)):
    train_data[i]['kps'] = np.array(train_data[i]['kps']).reshape(28*28)
for i in range(len(Xtest)):
    test_data[i]['kps'] = np.array(test_data[i]['kps']).reshape(28*28)


In [12]:
model.fit(Xtrain,ytrain)

1050/1050 [==============================] - 102s 97ms/step - loss: 0.1283 - accuracy: 0.9613


In [13]:
Xtest = Xtest.reshape(8400,28,28,1)

In [14]:
pred = model.predict(Xtest)
pred = np.argmax(pred,axis=1)
count = 0
for preds,ytests in zip(pred,ytest):
    if(preds == ytests):
        count+=1
(count/len(Xtest))*100

263/263 [==============================] - 4s 14ms/step


98.02380952380952

In [15]:
checks = [
    {
        'type':uptrain.Anomaly.DATA_DRIFT,
        'reference_dataset':'train_set.json',
        'is_embedding':False,
        'measurable_args':{
            'type': uptrain.MeasurableType.INPUT_FEATURE,
            'feature_name': 'kps'
        },
    }
]

In [23]:
training_args = {
    'training_func':model,
    'orig_training_file':'train_set.json'
}
evaluation_args = {
    'inference_func':tf.keras.metrics.Accuracy(),
    'golden_testing_dataset':'test_set.json'
}
cfg = {
    'checks':checks,
    'training_arg':training_args,
    'evaluation_args':evaluation_args,
    'retrain_after':200,
    'retraining_data':"uptrain_data"
}


In [24]:
framework = uptrain.Framework(cfg)

Deleting the folder:  uptrain_smart_data


In [40]:
for tests,labels in zip(Xtest,ytest):
    tests = tests.reshape(1,28,28,1)
    labels = labels.reshape(1,1)
    inputs = {"data": {"kps": tests}, "id": labels}
    #print(labels.shape)
    p = model.predict(tests)
    #print(p)
    prediction = np.array(np.argmax(p,axis=1))
    #print(prediction)
    #print(labels)
    idens = framework.log(inputs = inputs,outputs=prediction)
    if framework.version > 1:
        break


1/1 [==============================] - 0s 28ms/step


ValueError: object too deep for desired array

(1, 28, 28, 1)
